In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import logging
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from tqdm import tqdm

import torch
from transformers import (
  AdamW,
  ElectraConfig,
  ElectraTokenizer
)
from torch.utils.data import dataloader

# Change working directory
%cd /content/drive/MyDrive/my_ws/project/aischool-final/dialogLM

from dataloader.wellness import WellnessTextClassificationDataset
from model.koelectra import koElectraForSequenceClassification

logger = logging.getLogger(__name__)


# Model class mapping
MODEL_CLASSES ={
  "koelectra": (ElectraConfig, koElectraForSequenceClassification, ElectraTokenizer)
}

# Path to trained model checkpoint
CHECK_POINT ={
  "koelectra": "/content/drive/MyDrive/AISCHOOL9기-FINAL/koelectra/koelectra-wellness-text-classification-25-train.pth"
}

# Load model and tokenizer
def get_model_and_tokenizer(model_name, device):
  save_ckpt_path = CHECK_POINT[model_name]

  if model_name== "koelectra":
    model_name_or_path = "monologg/koelectra-base-discriminator"

    tokenizer = ElectraTokenizer.from_pretrained(model_name_or_path)
    electra_config = ElectraConfig.from_pretrained(model_name_or_path)
    model = koElectraForSequenceClassification.from_pretrained(pretrained_model_name_or_path=model_name_or_path,
                                                               config=electra_config,
                                                               num_labels=359)
  # Load checkpoint
  if os.path.isfile(save_ckpt_path):
      checkpoint = torch.load(save_ckpt_path, map_location=device)
      pre_epoch = checkpoint['epoch']
      # pre_loss = checkpoint['loss']
      model.load_state_dict(checkpoint['model_state_dict'], strict=False)

      print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}")

  return model, tokenizer

# Format model input
def get_model_input(data):
  if model_name== "koelectra":
    return {'input_ids': data['input_ids'],
              'attention_mask': data['attention_mask'],
              'labels': data['labels']
              }

# Evaluation function
def evaluate(model_name, device, batch_size, data_path):

  model, tokenizer = get_model_and_tokenizer(model_name, device)
  model.to(device)

  # Load evaluation dataset
  eval_dataset = WellnessTextClassificationDataset(file_path=data_path,device=device, tokenizer=tokenizer)

  # ❗ shuffle=True is not typical for evaluation but does not affect overall metrics like accuracy
  eval_dataloader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=True)

  logger.info("***** Running evaluation on %s dataset *****")
  logger.info("  Num examples = %d", len(eval_dataset))
  logger.info("  Batch size = %d", batch_size)

  loss = 0
  acc = 0

  model.eval()
  for data in tqdm(eval_dataloader, desc="Evaluating"):
    with torch.no_grad():
      inputs = get_model_input(data)
      outputs = model(**inputs)

      # ❗ Loss is accumulated without normalization per sample count.
      # This is not the exact average loss.
      loss += outputs[0]
      logit = outputs[1]
      # Count correct predictions and convert tensor to Python number
      acc += (logit.argmax(1)==inputs['labels']).sum().item()

  # Normalize loss and accuracy by total number of samples
  return loss / len(eval_dataset), acc / len(eval_dataset)

if __name__ == '__main__':
  # Path to evaluation data
  data_path = f"/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM/data/wellness_data_for_text_classification_test.txt"
  # Checkpoint path
  save_ckpt_path = f"/content/drive/MyDrive/AISCHOOL9기-FINAL/koelectra/koelectra-wellness-text-classification-25-train.pth"

  model_name_or_path = "monologg/koelectra-base-discriminator"

  batch_size = 16   # Batch size for evaluation

  ctx = "cuda" if torch.cuda.is_available() else "cpu"
  device = torch.device(ctx)

  model_names=["koelectra"]
  for model_name in model_names:
    eval_loss, eval_acc = evaluate(model_name, device, batch_size, data_path)
    print(f'\tLoss: {eval_loss:.4f}(valid)\t|\tAcc: {eval_acc * 100:.1f}%(valid)')

/content/drive/MyDrive/my_ws/project/aischool-final/dialogLM


Some weights of koElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


load pretrain from: /content/drive/MyDrive/AISCHOOL9기-FINAL/koelectra/koelectra-wellness-text-classification-25-train.pth, epoch=24


Evaluating: 100%|██████████| 66/66 [00:33<00:00,  2.00it/s]

	Loss: 0.2260(valid)	|	Acc: 32.0%(valid)
